In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, RNN, Lambda, Concatenate
from keras.layers.merge import Multiply
from keras.models import Model
from keras.losses import mean_squared_error, binary_crossentropy
from keras.models import Model
import keras.backend as K
from sklearn.metrics import mean_squared_error as mse

def save_model(model, model_str):
    model_json = model.to_json()
    with open('trained/'+model_str+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('trained/'+model_str+".h5")
    print("Saved model to disk")

Using TensorFlow backend.


In [2]:
df = pd.read_csv('data/bounces_x_1.csv')
df2 = pd.read_csv('data/sim_data_borders.csv')
mixdf = pd.concat([df, df2[:5000]])
mix = mixdf.to_numpy()
mix = mix.reshape(2000,5,4)
np.random.shuffle(mix)
mix_train_x = mix[:1800,:4,[0,2]]
mix_train_y = mix[:1800,4,:]
mix_test_x = mix[1800:,:4,[0,2]]
mix_test_y = mix[1800:,4,:]

In [3]:
# n2n model
inputs = Input(shape=(4,2))
x = LSTM(4, return_sequences=False)(inputs)
x = Dense(4, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(20)(x)
x = Dense(4)(x)
x = Dense(4)(x)
model_n2n = Model(inputs=inputs, outputs=x)
model_n2n.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
model_n2n.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 2)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 112       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_3 (Dense)              (None, 20)                180       
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 84        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 20        
Total para

In [4]:
model_n2n.fit(mix_train_x, mix_train_y, epochs=100)

Epoch 1/100
1800/1800 [==============================] - 1s 561us/step - loss: 0.1393 - acc: 0.6728
Epoch 2/100
1800/1800 [==============================] - 0s 154us/step - loss: 0.0372 - acc: 0.7444
Epoch 3/100
1800/1800 [==============================] - 0s 157us/step - loss: 0.0300 - acc: 0.7444
Epoch 4/100
1800/1800 [==============================] - 0s 173us/step - loss: 0.0207 - acc: 0.8228
Epoch 5/100
1800/1800 [==============================] - 0s 158us/step - loss: 0.0089 - acc: 0.9144
Epoch 6/100
1800/1800 [==============================] - 0s 158us/step - loss: 0.0063 - acc: 0.9139
Epoch 7/100
1800/1800 [==============================] - 0s 164us/step - loss: 0.0057 - acc: 0.9194
Epoch 8/100
1800/1800 [==============================] - 0s 162us/step - loss: 0.0055 - acc: 0.9228
Epoch 9/100
1800/1800 [==============================] - 0s 155us/step - loss: 0.0053 - acc: 0.9244
Epoch 10/100
1800/1800 [==============================] - 0s 165us/step - loss: 0.0052 - acc: 0.9211

1800/1800 [==============================] - 0s 161us/step - loss: 0.0012 - acc: 0.9761
Epoch 83/100
1800/1800 [==============================] - 0s 159us/step - loss: 0.0012 - acc: 0.9744
Epoch 84/100
1800/1800 [==============================] - 0s 162us/step - loss: 0.0012 - acc: 0.9761
Epoch 85/100
1800/1800 [==============================] - 0s 161us/step - loss: 0.0012 - acc: 0.9778
Epoch 86/100
1800/1800 [==============================] - 0s 161us/step - loss: 0.0012 - acc: 0.9756
Epoch 87/100
1800/1800 [==============================] - 0s 164us/step - loss: 0.0012 - acc: 0.9783
Epoch 88/100
1800/1800 [==============================] - 0s 160us/step - loss: 0.0012 - acc: 0.9761
Epoch 89/100
1800/1800 [==============================] - 0s 159us/step - loss: 0.0012 - acc: 0.9767
Epoch 90/100
1800/1800 [==============================] - 0s 159us/step - loss: 0.0012 - acc: 0.9767
Epoch 91/100
1800/1800 [==============================] - 0s 158us/step - loss: 0.0012 - acc: 0.9783
Epo

In [5]:
mse(mix_test_y, model_n2n.predict(mix_test_x))**0.5

0.03724479412322676